# EndToEnd TalkingBot on PC client (Windows)

> make sure you are running in a conda environment with Python 3.10

[Intel® Extension for Transformers Neural Chat](https://github.com/intel/intel-extension-for-transformers/tree/main/intel_extension_for_transformers/neural_chat) provides a lot of plugins to meet different users' scenarios. In this notebook we will show you how to create a TalkingBot on your local laptop with **Intel CPU** (no GPU needed).

Behind the scene, a TalkingBot is composed of a pipeline of
1. recognize user's prompt audio and convert to text
2. text understanding and question answering by Large Language Models
2. convert answer text to speech

This is a notebook to let you know how to create such a TalkingBot on PC.

## Audio To Text

In [ ]:
!curl -O https://raw.githubusercontent.com/intel/intel-extension-for-transformers/main/intel_extension_for_transformers/neural_chat/assets/audio/sample.wav

In [ ]:
from intel_extension_for_transformers.neural_chat.pipeline.plugins.audio.asr import AudioSpeechRecognition

In [ ]:
from IPython.display import Audio
Audio(r"./sample.wav", rate=16000)

In [ ]:
asr = AudioSpeechRecognition(model_name_or_path="openai/whisper-tiny")

In [ ]:
text = asr.audio2text(r"./sample.wav")
print(text)

## LLM

### Directly load given int4 model to do inference

Here for quick demo, we just use a given int4 model to generate text. If you want to convert your int4 model manually, please refer to next cell.

In [ ]:
from transformers import AutoTokenizer, TextStreamer
from intel_extension_for_transformers.transformers import AutoModelForCausalLM
model_name = "meta-llama/Llama-2-7b-chat-hf"     # Hugging Face model_id or local model
prompt = text

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
streamer = TextStreamer(tokenizer)

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300)

### Convert int4 model to do inference

In [ ]:
from intel_extension_for_transformers.neural_chat import build_chatbot, PipelineConfig
from intel_extension_for_transformers.transformers import WeightOnlyQuantConfig
from intel_extension_for_transformers.neural_chat.config import LoadingModelConfig
config = PipelineConfig(optimization_config=WeightOnlyQuantConfig(compute_dtype="int8", weight_dtype="int4"), 
                        loading_config=LoadingModelConfig(use_neural_speed=True),
                        model_name_or_path="meta-llama/Llama-2-7b-chat-hf")
chatbot = build_chatbot(config)
response = chatbot.predict(query="who is pat gelsinger")
print(response)

## Text To Speech

In [ ]:
from intel_extension_for_transformers.neural_chat.pipeline.plugins.audio.tts import TextToSpeech

In [ ]:
tts = TextToSpeech()

In [ ]:
result_path = tts.text2speech("Hello there, I am your Talking Bot!", "output.wav")

In [ ]:
from IPython.display import Audio
Audio(result_path, rate=16000)